In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import os
import pickle

#### Basic overview of generated embedding .pkl file

In [ ]:
def load_pickle(filepath: str):
    """Load a pickled object.

    Args:
        filepath (str): Path to pickle (.pkl) file.

    Returns:
        Any: Loaded object.
    """
    with open(filepath, "rb") as f:
        data = pickle.load(f)
    return data

In [ ]:
embeddings = load_pickle("../outputs/processed_data/mmfair_feat.pkl")
emb_old = load_pickle("../outputs/prev_data/mmfair_feat.pkl")
cols = load_pickle("../outputs/processed_data/mmfair_cols.pkl")

In [ ]:
print(len(embeddings.keys()), len(emb_old.keys()))
### Get keys in embeddings but not in embeddings_old
keys = set(embeddings.keys()) - set(emb_old.keys())
print(len(keys))
### Get keys in embeddings_old but not in embeddings
keys = set(emb_old.keys()) - set(embeddings.keys())
print(len(keys))

In [ ]:
risk_dict = load_pickle("..\outputs\evaluation\ext_stay_7_concat_static_timeseries\pf_ext_stay_7_concat_static_timeseries.pkl")

In [ ]:
fair_dict = load_pickle("../outputs/fairness/ext_stay_7_None_timeseries/pf_ext_stay_7_None_timeseries.pkl")
fair_dict_cst = load_pickle("../outputs/fairness/ext_stay_7_concat_static_timeseries/pf_ext_stay_7_concat_static_timeseries.pkl")
fair_dict_cstn = load_pickle("../outputs/fairness/ext_stay_7_concat_static_timeseries_notes/pf_ext_stay_7_concat_static_timeseries_notes.pkl")

In [ ]:
fair_dict

In [ ]:
### Select the keys that start with fair_
fair_keys = [key for key in fair_dict.keys() if key.startswith("fair_")]
### Select items from the dictionary that start with fair_
fair_dict_f = {key: fair_dict[key] for key in fair_keys}
fair_dict_cstf = {key: fair_dict_cst[key] for key in fair_keys}
fair_dict_cstnf = {key: fair_dict_cstn[key] for key in fair_keys}
### Display values within the keys
fair_df = pd.concat([pd.DataFrame(fair_dict_f), pd.DataFrame(fair_dict_cstf), pd.DataFrame(fair_dict_cstnf)], axis=1)

In [ ]:
fair_df.T

In [ ]:
len(risk_dict['risk_quantile']), len(risk_dict['y_prob']), len(risk_dict['test_ids'])

In [ ]:
risk_dict['yd_idx']

In [ ]:
embeddings

In [ ]:
cols

In [ ]:
embeddings[10000032]['notes'][0]

In [ ]:
for attr_i in range(len(cols['notes_cols'])):
    for i in range(len(embeddings[16486866]['notes'])):
        print(f"Note:{embeddings[16486866]['notes'][i][0]}, Score:{embeddings[16486866]['notes'][i][1]}")

In [ ]:
for attr_i in range(len(cols['dynamic1_cols'])):
    for i in range(len(embeddings[16486866]['dynamic_1'])):
        if 'creatinine' in cols['dynamic1_cols'][attr_i]:
            print(f"Col:{cols['dynamic1_cols'][attr_i]}={embeddings[16486866]['dynamic_1'][i][attr_i]}")

In [ ]:
for attr_i in range(len(cols['dynamic0_cols'])):
    for i in range(len(embeddings[16486866]['dynamic_0'])):
        print(f"Col:{cols['dynamic0_cols'][attr_i]}={embeddings[16486866]['dynamic_0'][i][attr_i]}")

In [ ]:
for attr_i in range(len(cols['static_cols'])):
    print(f"Col:{cols['static_cols'][attr_i]}={embeddings[16486866]['static'][0][attr_i]}")


#### Recode some of the embeddings

In [ ]:
for id_val in embeddings.keys():
    embeddings[id_val]['static'] = emb_new[id_val]['static']
    embeddings[id_val]['dynamic_0'] = emb_new[id_val]['dynamic_0']
    embeddings[id_val]['dynamic_1'] = emb_new[id_val]['dynamic_1']
    embeddings[id_val]['in_hosp_death'] = emb_new[id_val]['in_hosp_death']
    embeddings[id_val]['ext_stay_7'] = emb_new[id_val]['ext_stay_7']
    embeddings[id_val]['icu_admission'] = emb_new[id_val]['icu_admission']
    embeddings[id_val]['non_home_discharge'] = emb_new[id_val]['non_home_discharge']

In [ ]:
with open(os.path.join("../outputs/processed_data", "mmfair_feat.pkl"), "wb") as f:
        pickle.dump(embeddings, f)

In [ ]:
extr_list = []
for item in embeddings[id_val]['notes']:
    extr_list.append(item[1])
print(extr_list)

In [ ]:
embeddings[10000032]['static'].shape[1]

In [ ]:
embeddings[list(embeddings.keys())[-1]]

In [ ]:
list(embeddings.keys())

In [ ]:
embeddings[list(embeddings.keys())[0]].keys()

In [ ]:
[key for key in embeddings[list(embeddings.keys())[0]].keys() if "dynamic" in key]

#### Test training IDs

In [ ]:
train_hosp_death = pd.read_csv('../outputs/prep_data/training_ids_in_hosp_death.csv')
val_hosp_death = pd.read_csv('../outputs/prep_data/validation_ids_in_hosp_death.csv')
test_hosp_death = pd.read_csv('../outputs/prep_data/testing_ids_in_hosp_death.csv')

In [ ]:
train_icu = pd.read_csv('../outputs/prep_data/training_ids_icu_admission.csv')

In [ ]:
overlap = set(train_icu['subject_id']).intersection(set(train_hosp_death['subject_id']))
print(len(overlap))

In [ ]:
emb_ids = list(embeddings.keys())
overlap = set(emb_ids).intersection(set(train_hosp_death['subject_id']))
overlapv = set(emb_ids).intersection(set(val_hosp_death['subject_id']))
overlapt = set(emb_ids).intersection(set(test_hosp_death['subject_id']))
print(len(overlap), len(overlapv), len(overlapt), len(train_hosp_death), len(embeddings.keys()))